In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Concatenate, Conv2DTranspose, MaxPool2D, Flatten, Conv2D, Reshape, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from keras.callbacks import Callback
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import LeakyReLU, BatchNormalization, UpSampling2D, concatenate, Activation
from tensorflow.keras.models import load_model


from tensorflow.keras.layers import GlobalMaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Conv2DTranspose, Input

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, Dropout, Concatenate, LeakyReLU, BatchNormalization, Add
from tensorflow.keras.initializers import he_normal

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib import cm 
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
import matplotlib.colors as mcolors
from matplotlib.colors import BoundaryNorm
import cartopy.crs as ccrs
import cartopy.feature as cfeature
plt.style.use("dark_background")

from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import MinMaxScaler

# scaler = MaxAbsScaler()
scaler = MinMaxScaler(feature_range=(0,1))

tf.random.set_seed(42)  # to replicate results

2024-08-21 13:17:40.624109: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-21 13:17:40.624150: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-21 13:17:40.625311: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-21 13:17:40.632165: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-21 13:17:41.815654: W tensorflow/compiler/tf2

In [2]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  4


In [3]:
# Get the list of available physical GPUs
gpus = tf.config.list_physical_devices('GPU')

# Set memory growth for each GPU
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


2024-08-21 13:17:45.107832: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 75826 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:03:00.0, compute capability: 8.0
2024-08-21 13:17:45.109348: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 75826 MB memory:  -> device: 1, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:44:00.0, compute capability: 8.0
2024-08-21 13:17:45.110751: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 75826 MB memory:  -> device: 2, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:84:00.0, compute capability: 8.0
2024-08-21 13:17:45.112145: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 75826 MB memory:  -> device: 3, name: NVIDIA A100-SXM4-80GB, pci bu

In [4]:
diri = '/work/bb0983/athul_satheesh/e_obs_precip/'
diro = '/work/bb0983/athul_satheesh/downscaled_data/europe/figures/'

coarse_raw = 'rr_ens_mean_1.0deg_reg_v29.0e.nc'
fine_raw = 'rr_ens_mean_0.1deg_reg_v29.0e.nc'

lati = 43#40
latf = 59#60

loni = -6#-10
lonf = 15#30

strt = '1950-01-01'
last = '2023-12-31'

coarse_data = xr.open_dataset(diri+coarse_raw).rr.transpose('time','lat','lon').sel(time=slice(strt, last), 
                                                                                    lat=slice(lati, latf), 
                                                                                    lon=slice(loni, lonf)
                                                                                   )
fine_data = xr.open_dataset(diri+fine_raw).rr.transpose('time','latitude','longitude').sel(time=slice(strt, last), 
                                                                                           latitude=slice(lati, latf), 
                                                                                           longitude=slice(loni, lonf)
                                                                                          )
fine_data = fine_data.rename({'latitude':'lat', 'longitude':'lon'})

In [5]:
dims_coarse = coarse_data.shape
dims_fine = fine_data.shape

In [6]:
dims_coarse, dims_fine

((27028, 16, 21), (27028, 160, 210))

In [7]:
train_strt = strt
train_last = '2000-12-31'

test_strt = '2001-01-01'
test_last = last

coarse_data_train = coarse_data.sel(time=slice(train_strt, train_last))#.interp(lat=fine_data.lat, lon=fine_data.lon, method='linear')
# coarse_data_train = coarse_data_train.fillna(coarse_data_train.mean())
coarse_data_test = coarse_data.sel(time=slice(test_strt, test_last))#.interp(lat=fine_data.lat, lon=fine_data.lon, method='linear')
# coarse_data_test = coarse_data_test.fillna(coarse_data_test.mean())
# print(coarse_data_train.shape, coarse_data_test.shape)

fine_data_train = fine_data.sel(time=slice(train_strt, train_last))
fine_data_test = fine_data.sel(time=slice(test_strt, test_last))
# print(fine_data_train.shape, fine_data_test.shape)

In [8]:
coarse_nan_mask = np.isnan(coarse_data_test)
fine_nan_mask = np.isnan(fine_data_test)

fill_val = -1e-10

In [9]:
coarse_data_train = coarse_data_train.fillna(fill_val)#.interp(lat=fine_data.lat, lon=fine_data.lon, method='linear')
# coarse_data_train = coarse_data_train.fillna(coarse_data_train.mean())
coarse_data_test = coarse_data_test.fillna(fill_val)#.interp(lat=fine_data.lat, lon=fine_data.lon, method='linear')
# coarse_data_test = coarse_data_test.fillna(coarse_data_test.mean())
# print(coarse_data_train.shape, coarse_data_test.shape)

fine_data_train = fine_data_train.fillna(fill_val)
fine_data_test = fine_data_test.fillna(fill_val)
# print(fine_data_train.shape, fine_data_test.shape)

In [10]:
fine_data_test.shape

(8400, 160, 210)

In [11]:
def ssim_loss(fake_images, real_images):
    """
    Computes the SSIM loss between fake and real images.
    
    Parameters:
        fake_images (tf.Tensor): Generated images.
        real_images (tf.Tensor): Real images.
    
    Returns:
        tf.Tensor: SSIM loss.
    """
    ssim_index = tf.image.ssim(fake_images, real_images, max_val=1.0)
    return 1 - tf.reduce_mean(ssim_index)

def generator_loss(fake_output, fake_images, real_images, penalty_weight=15, ssim_weight=15):
    """
    Generator loss function using Wasserstein loss with an added penalty term and SSIM loss.
    
    Parameters:
        fake_output (tf.Tensor): Output of the discriminator when given generated images.
        fake_images (tf.Tensor): Generated images.
        real_images (tf.Tensor): Real images.
        penalty_weight (float): Weight of the penalty term.
        ssim_weight (float): Weight of the SSIM loss term.
    
    Returns:
        tf.Tensor: Generator loss.
    """
    wasserstein_loss = -tf.reduce_mean(fake_output)
    
    # Penalty term for deviation from real outputs
    # penalty = penalty_weight * tf.reduce_mean(tf.abs(fake_output - real_output))
    penalty = penalty_weight * tf.reduce_mean(tf.abs(fake_images - real_images))
    
    # SSIM loss
    ssim_loss_value = ssim_loss(fake_images, real_images)
    
    return wasserstein_loss + penalty + ssim_weight * ssim_loss_value
    # return wasserstein_loss + ssim_weight * ssim_loss_value

def discriminator_loss(real_output, fake_output):
    """
    Discriminator loss function using Wasserstein loss.
    
    Parameters:
        real_output (tf.Tensor): Output of the discriminator when given real images.
        fake_output (tf.Tensor): Output of the discriminator when given generated images.
    
    Returns:
        tf.Tensor: Discriminator loss.
    """
    return tf.reduce_mean(fake_output) - tf.reduce_mean(real_output)

In [12]:
# Weight clipping
def clip_weights(model, clip_value):
    """
    Clips the weights of the model to be within the range [-clip_value, clip_value].
    
    Parameters:
        model (tf.keras.Model): The model whose weights will be clipped.
        clip_value (float): The value to clip the weights to.
    """
    for layer in model.layers:
        if hasattr(layer, 'kernel'):
            kernel = layer.kernel
            clipped_kernel = tf.clip_by_value(kernel, -clip_value, clip_value)
            layer.kernel.assign(clipped_kernel)

In [13]:
epoch_in = 100

model_diri = '/work/bb0983/athul_satheesh/downscaled_data/europe/trained_models/wgan_gp_extended_eu/' #'/work/bb0983/athul_satheesh/downscaled_data/europe/trained_models/wgan_gp/'
gen_model_fili = f'generator_det_epoch_{epoch_in}_adamV2.keras'
dis_model_fili = f'discriminator_det_epoch_{epoch_in}_adamV2.keras'

In [14]:
input_shape  = dims_coarse[1:] + (1,)
output_shape = dims_fine[1:] + (1,)

# lr = 0.5e-5
gen_lr = 1e-4 # updated to 1e-4 from 2e-4 100 epochs
dis_lr = 0.5e-5 # updated to 0.5e-5 from 2e-4 after 100 epochs
clip_value = 1e-3


with strategy.scope():
    
    # gen_optimizer = RMSprop(learning_rate=lr, momentum=0.5)
    # dis_optimizer = RMSprop(learning_rate=lr, momentum=0.5)
    gen_optimizer = Adam(learning_rate=gen_lr)
    dis_optimizer = Adam(learning_rate=dis_lr)

    # Load the generator and discriminator models
    gen = load_model(model_diri+gen_model_fili)
    dis = load_model(model_diri+dis_model_fili)    

    gen.compile(optimizer=gen_optimizer,)
    dis.compile(optimizer=dis_optimizer,)

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [15]:
gen.summary()

Model: "generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 generator_input (InputLaye  [(None, 16, 21, 1)]       0         
 r)                                                              
                                                                 
 conv_4_conv (Conv2D)        (None, 16, 21, 16)        160       
                                                                 
 conv_4_lrelu (LeakyReLU)    (None, 16, 21, 16)        0         
                                                                 
 conv_4_bn (BatchNormalizat  (None, 16, 21, 16)        64        
 ion)                                                            
                                                                 
 conv_3_conv (Conv2D)        (None, 16, 21, 32)        4640      
                                                                 
 conv_3_lrelu (LeakyReLU)    (None, 16, 21, 32)        0 

In [16]:
dis.summary()

Model: "wGAN_discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 discriminator_input (Input  [(None, 160, 210, 1)]     0         
 Layer)                                                          
                                                                 
 conv2_conv (Conv2D)         (None, 80, 105, 128)      1280      
                                                                 
 conv2_lrelu (LeakyReLU)     (None, 80, 105, 128)      0         
                                                                 
 conv2_bn (BatchNormalizati  (None, 80, 105, 128)      512       
 on)                                                             
                                                                 
 conv4_conv (Conv2D)         (None, 80, 105, 256)      295168    
                                                                 
 conv4_lrelu (LeakyReLU)     (None, 80, 105, 256

In [17]:
# Convert data to tensor
def preprocess_data(data):
    data = tf.convert_to_tensor(data.values, dtype=tf.float32)
    return tf.reshape(data, data.shape + (1,))
# Compute and update gradients
def train_step(coarse_data_batch, fine_data_batch, dis, clip_value):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as dis_tape:
        fake_data_batch = gen(coarse_data_batch, training=True)
        real_output = dis(fine_data_batch, training=True)
        fake_output = dis(fake_data_batch, training=True)
        gen_loss = generator_loss(fake_output, fake_data_batch, fine_data_batch)
        dis_loss = discriminator_loss(real_output, fake_output)

    gradients_of_gen = gen_tape.gradient(gen_loss, gen.trainable_variables)
    gradients_of_dis = dis_tape.gradient(dis_loss, dis.trainable_variables)
    
    gen_optimizer.apply_gradients(zip(gradients_of_gen, gen.trainable_variables))
    dis_optimizer.apply_gradients(zip(gradients_of_dis, dis.trainable_variables))
    
    # Clip discriminator weights
    clip_weights(dis, clip_value)
    
    return gen_loss, dis_loss

@tf.function
def distributed_train_step(coarse_data_batch, fine_data_batch, dis, clip_value):
    per_replica_gen_losses, per_replica_dis_losses = strategy.run(train_step, args=(coarse_data_batch, fine_data_batch, dis, clip_value))
    mean_gen_loss = strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_gen_losses, axis=None)
    mean_dis_loss = strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_dis_losses, axis=None)
    return mean_gen_loss, mean_dis_loss

def train_gan(gen, dis, coarse_data_train, fine_data_train, clip_value, epochs, batch_size, save_intermediate=True):
    dataset = tf.data.Dataset.from_tensor_slices((preprocess_data(coarse_data_train), preprocess_data(fine_data_train)))
    # dataset = dataset.shuffle(buffer_size=1024).batch(batch_size).repeat(epochs)
    dataset = dataset.shuffle(buffer_size=365*2).batch(batch_size)#.repeat(epochs)
    distributed_dataset = strategy.experimental_distribute_dataset(dataset)

    for epoch in range(epochs):
        total_gen_loss = 0.0
        total_dis_loss = 0.0
        num_batches = 0

        for coarse_data_batch, fine_data_batch in distributed_dataset:
            gen_loss, dis_loss = distributed_train_step(coarse_data_batch, fine_data_batch, dis, clip_value)
            total_gen_loss += gen_loss
            total_dis_loss += dis_loss
            num_batches += 1

        avg_gen_loss = total_gen_loss / num_batches
        avg_dis_loss = total_dis_loss / num_batches
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch + 1}, Generator Loss: {avg_gen_loss:.5f}, Discriminator Loss: {avg_dis_loss:.5f}")

        # Save models every 100 epochs
        if save_intermediate:
            if (epoch + 1) % 50 == 0:
                gen.save(f'/work/bb0983/athul_satheesh/downscaled_data/europe/trained_models/wgan_gp_extended_eu/generator_det_epoch_{epoch_in+epoch+1}_adamV2.keras')
                dis.save(f'/work/bb0983/athul_satheesh/downscaled_data/europe/trained_models/wgan_gp_extended_eu/discriminator_det_epoch_{epoch_in+epoch+1}_adamV2.keras')

In [18]:
# def normalize_data(dataset, scaler, train=True):
    
#     dims = dataset.shape
    
#     dataset = dataset.values.reshape(-1, dims[1]*dims[2])

#     if train:
#         normalized_data = scaler.fit_transform(dataset)
#     else:
#         normalized_data = scaler.transform(dataset)
        
#     normalized_data = normalized_data.reshape(dims + (1,) ) # Make 4D
    
#     return normalized_data

In [19]:
epochs = 450 
batch_size = 365

In [20]:
%%time
train_gan(gen, dis, coarse_data_train, fine_data_train, clip_value, epochs, batch_size, save_intermediate=True)

2024-08-21 13:18:38.283251: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


INFO:tensorflow:Collective all_reduce tensors: 50 all_reduces, num_devices = 4, group_size = 4, implementation = CommunicationImplementation.NCCL, num_packs = 1
INFO:tensorflow:Collective all_reduce tensors: 14 all_reduces, num_devices = 4, group_size = 4, implementation = CommunicationImplementation.NCCL, num_packs = 1
INFO:tensorflow:Collective all_reduce tensors: 50 all_reduces, num_devices = 4, group_size = 4, implementation = CommunicationImplementation.NCCL, num_packs = 1
INFO:tensorflow:Collective all_reduce tensors: 14 all_reduces, num_devices = 4, group_size = 4, implementation = CommunicationImplementation.NCCL, num_packs = 1


2024-08-21 13:18:53.966073: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inreplica_1/wGAN_discriminator/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-08-21 13:18:55.570090: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-08-21 13:18:55.603677: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-08-21 13:18:55.644402: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-08-21 13:18:55.658132: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-08-21 13:18:55.686576: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-08-21 13:21:14.066352: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fd882d0

INFO:tensorflow:Collective all_reduce tensors: 50 all_reduces, num_devices = 4, group_size = 4, implementation = CommunicationImplementation.NCCL, num_packs = 1
INFO:tensorflow:Collective all_reduce tensors: 14 all_reduces, num_devices = 4, group_size = 4, implementation = CommunicationImplementation.NCCL, num_packs = 1


2024-08-21 13:21:42.851053: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inreplica_1/wGAN_discriminator/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


Epoch 10, Generator Loss: 36.92801, Discriminator Loss: -1.41284
Epoch 20, Generator Loss: 37.16657, Discriminator Loss: -1.50220
Epoch 30, Generator Loss: 37.32770, Discriminator Loss: -1.51201
Epoch 40, Generator Loss: 36.43479, Discriminator Loss: -1.52489
Epoch 50, Generator Loss: 35.96231, Discriminator Loss: -1.45975
Epoch 60, Generator Loss: 34.86019, Discriminator Loss: -1.50837
Epoch 70, Generator Loss: 34.89820, Discriminator Loss: -1.53198
Epoch 80, Generator Loss: 34.38742, Discriminator Loss: -1.46730
Epoch 90, Generator Loss: 34.30152, Discriminator Loss: -1.58649
Epoch 100, Generator Loss: 33.96468, Discriminator Loss: -1.58204
Epoch 110, Generator Loss: 34.95599, Discriminator Loss: -1.59402
Epoch 120, Generator Loss: 33.75338, Discriminator Loss: -1.56530
Epoch 130, Generator Loss: 33.36537, Discriminator Loss: -1.64474
Epoch 140, Generator Loss: 34.01175, Discriminator Loss: -1.58774
Epoch 150, Generator Loss: 33.15188, Discriminator Loss: -1.53760
Epoch 160, Generato


KeyboardInterrupt



In [21]:
# def inverse_normalize_data(normalized_data, scaler):
    
#     dims = normalized_data.shape
    
#     normalized_data = normalized_data.reshape(-1, dims[1]*dims[2])
    
#     original_data = scaler.inverse_transform(normalized_data)
    
#     original_data = original_data.reshape(dims)
#     return original_data

In [22]:
downscaled_data = gen.predict( coarse_data_test.values.reshape( coarse_data_test.shape + (1,) ) )

Exception ignored in: <function AtomicFunction.__del__ at 0x7ffb412f89d0>
Traceback (most recent call last):
  File "/home/b/b382999/.conda/envs/ml_env/lib/python3.9/site-packages/tensorflow/python/eager/polymorphic_function/atomic_function.py", line 286, in __del__
    def __del__(self):
KeyboardInterrupt: 

KeyboardInterrupt



In [ ]:
downscaled_data.shape

In [ ]:
# downscaled_data = inverse_normalize_data(downscaled_data, scaler)

In [ ]:
downscaled_data = xr.DataArray(name='precipitation', data=downscaled_data.squeeze(), 
                               dims=fine_data_test.dims, coords=fine_data_test.coords, 
                               attrs=fine_data_test.attrs)

In [ ]:
downscaled_data = downscaled_data.where(~fine_nan_mask, np.nan)
fine_data_test = fine_data_test.where(~fine_nan_mask, np.nan)
coarse_data_test = coarse_data_test.where(~coarse_nan_mask, np.nan)

In [ ]:
diro = '/work/bb0983/athul_satheesh/downscaled_data/europe/downscaled_data/'
# downscaled_data.to_netcdf(diro + 'e_obs_eu_downscaled_wgan_transpose.nc')

In [ ]:
# save model
# gen.save('/work/bb0983/athul_satheesh/downscaled_data/europe/trained_models/wgan_gp_generator_transpose.keras')

In [ ]:
# save model
# dis.save('/work/bb0983/athul_satheesh/downscaled_data/europe/trained_models/wgan_gp_discriminator_transpose.keras')

In [ ]:
date1 = '2021-06-15'
date2 = '2021-09-15'

print(f"MAE: {abs(fine_data_test - downscaled_data).mean().values:.4f}")
print(f"RMSE: {np.sqrt( ( (fine_data_test - downscaled_data)**2 ).mean() ).values:.4f}")

In [ ]:
# np.corrcoef(fine_data_test.values.flatten(), downscaled_data.values.flatten())[0,1]

In [ ]:
from sklearn.metrics import r2_score
def compute_r2_with_nans(y_true, y_pred):
    # Mask for non-NaN values
    mask = ~np.isnan(y_true) & ~np.isnan(y_pred)
    
    # Filter out NaN values
    y_true_filtered = y_true[mask]
    y_pred_filtered = y_pred[mask]
    
    # Compute R2 score
    r2 = r2_score(y_true_filtered, y_pred_filtered)
    return r2
r2 = compute_r2_with_nans(fine_data_test.values.flatten(), downscaled_data.values.flatten())
r2

In [ ]:
# figs, axx = plt.subplots(figsize=(8,6))
# axx.scatter(fine_data_test.values.flatten(), downscaled_data.values.flatten(), s=0.1, alpha=0.9, color='C0')
# axx.plot([0,1000], [0,1000], color='red', alpha=0.9, zorder=1, ls='--')
# axx.set_xlim(-8,500)
# axx.set_ylim(-8,500)
# axx.set_xlabel('Observation', size=15)
# axx.set_ylabel('Downscaled', size=15)
# axx.text(45,400,f'$R^2: {r2:.2f}$', color='white', size=12)
# axx.grid(True, alpha=0.2, color='C1')

In [ ]:
# Define the colormap colors
colors_blues2black = [(0, 0, 0), (0, 0.90, 1)]  # Blue to Black
colors_RdBlBu = [(1, 0, 0), (0, 0, 0), (0, 0.90, 1)]  # Blue to Black

# Create the colormap
cmap_name = 'BluesToBlack'
blues_to_black = mcolors.LinearSegmentedColormap.from_list(cmap_name, colors_blues2black)
Rd_bl_Bu = mcolors.LinearSegmentedColormap.from_list('RdBlBu', colors_RdBlBu)


fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(12,10), sharex=True, sharey=True,
                        subplot_kw=dict(projection=ccrs.PlateCarree(), facecolor='black'),
                        gridspec_kw={'wspace': -0.05, 'hspace': -0.45})

# plots = [coarse_data_test, fine_data_test, downscaled_data, (fine_data_test - downscaled_data)]
plots = [coarse_data_test, fine_data_test, downscaled_data, abs(fine_data_test - downscaled_data)]
cmap = [blues_to_black, blues_to_black, blues_to_black, 'afmhot']
# cmap = [blues_to_black, blues_to_black, blues_to_black, Rd_bl_Bu]
# levels = [np.arange(0,30.25,0.25), np.arange(0,30.25,0.25), np.arange(0,30.25,0.25), np.arange(-12,12.25,0.25)]
levels = [np.arange(0,6.125,0.125), np.arange(0,6.125,0.125), np.arange(0,6.125,0.125), np.arange(0,6.125,0.125)]
# title = ['a) Coarse','b) Fine','c) Downscaled','d) Observed-Downscaled']
title = ['a) Coarse','b) Fine','c) Downscaled','d) MAE']
# ticks = [np.arange(0,33,3), np.arange(0,33,3), np.arange(0,33,3), np.arange(-12,13,4)]
ticks = [np.arange(0,7,1), np.arange(0,7,1), np.arange(0,7,1), np.arange(0,7,1)]

for i, ax in enumerate(axs.flatten()):
    plots[i].sel(time=slice(date1, date2)).mean('time').plot(cmap=cmap[i], levels=levels[i], 
                                                             ax=ax, cbar_kwargs={'orientation':'vertical',
                                                                                'pad':0.01, 'label':'',
                                                                                'shrink':0.48, 'drawedges':False,
                                                                                'ticks': ticks[i], },
                                                            alpha=0.90)
    
    ax.text(0, 55.2, f'{title[i]}',size=13, color='white')
    ax.coastlines(linewidth=1.5, color='white')
    ax.add_feature(cfeature.BORDERS.with_scale('50m'), linewidth=1.5, edgecolor='white');
    ax.patch.set_facecolor('black')
plt.show()